In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
import sys
sys.path.append('../')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig,\
                        TBConfig, PUSCHPilotPattern, TBEncoder, PUSCHPrecoder, LayerMapper, LayerDemapper, check_pusch_configs,\
                        TBDecoder, PUSCHLSChannelEstimator
from sionna.nr.utils import generate_prng_seq, select_mcs, calculate_tb_size
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, \
    BinarySource, config_parser, fft_size_return
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector,\
        LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper,\
        ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement
from sionna.mapping import Mapper, Demapper
from sionna.fec.ldpc import LDPC5GEncoder

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import pickle
from collections import namedtuple
import json
from tqdm.notebook import tqdm
import itertools
import io
import h5py

## A Hello World Example

Let us start with a simple "Hello, World!" example in which we will simulate PUSCH transmissions from a single transmitter to a single receiver over an AWGN channel.

In [3]:
# _num_tx = 1
# _num_rx = 1
# _num_tx_ant = 1
# _num_rx_ant = 8
# _carrier_frequency = 2.55e9  # Carrier frequency in Hz.
# _link_direction = "uplink"

# # Configure antenna arrays


# _channel_model_0 = CDL("C", 150e-9, _carrier_frequency, _ue_antenna, _gnb_array, _link_direction, min_speed=10)

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class SystemConfig:
    NCellId: int = 246
    FrequencyRange: int = 1
    BandWidth: int = 100
    Numerology: int = 1
    CpType: int = 0
    NTxAnt: int = 1
    NRxAnt: int = 8
    BwpNRb: int = 273
    BwpRbOffset: int = 0
    harqProcFlag: int = 0
    nHarqProc: int = 1
    rvSeq: int = 0


@dataclass
class UeConfig:
    rvIdx: int = 0
    TransformPrecoding: int = 0
    Rnti: int = 20002
    nId: int = 246
    CodeBookBased: int = 0
    DmrsPortSetIdx: List[int] = field(default_factory=lambda: [0])  # FIXED
    NLayers: int = 1
    NumDmrsCdmGroupsWithoutData: int = 2
    Tpmi: int = 0
    FirstSymb: int = 0
    NPuschSymbAll: int = 14
    RaType: int = 1
    FirstPrb: int = 31
    NPrb: int = 4
    FrequencyHoppingMode: int = 0
    McsTable: int = 0
    Mcs: int = 3
    ILbrm: int = 0
    nScId: int = 0
    NnScIdId: int = 246
    DmrsConfigurationType: int = 0
    DmrsDuration: int = 1
    DmrsAdditionalPosition: int = 1
    PuschMappingType: int = 0
    DmrsTypeAPosition: int = 3
    HoppingMode: int = 0
    NRsId: int = 0
    Ptrs: int = 0
    ScalingFactor: int = 0
    OAck: int = 0
    IHarqAckOffset: int = 11
    OCsi1: int = 0
    ICsi1Offset: int = 7
    OCsi2: int = 0
    ICsi2Offset: int = 0
    NPrbOh: int = 0
    nCw: int = 1
    TpPi2Bpsk: int = 0

@dataclass
class MyConfig:
    Sys: SystemConfig
    Ue: List[UeConfig]
    Num_tx: int = 1
    Num_rx: int = 1
    Carrier_frequency: float = 2.55e9  # Carrier frequency in Hz

# Example usage
My_Config = MyConfig(SystemConfig(), [UeConfig()])


In [5]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self, My_Config: MyConfig):
        self.My_Config = My_Config
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=My_Config.Sys.NCellId,
                cyclic_prefix="normal" if ~My_Config.Sys.CpType else "extended",
                subcarrier_spacing=15*(2**My_Config.Sys.Numerology),
                n_size_grid=My_Config.Sys.BwpNRb,
                n_start_grid=My_Config.Sys.BwpRbOffset,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=My_Config.Ue[0].DmrsConfigurationType + 1,
                length=My_Config.Ue[0].DmrsDuration,
                additional_position=My_Config.Ue[0].DmrsAdditionalPosition,
                dmrs_port_set=My_Config.Ue[0].DmrsPortSetIdx,
                n_id=My_Config.Ue[0].NnScIdId,
                n_scid=My_Config.Ue[0].nScId,
                num_cdm_groups_without_data=My_Config.Ue[0].NumDmrsCdmGroupsWithoutData,
                type_a_position=My_Config.Ue[0].DmrsTypeAPosition
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=My_Config.Ue[0].nId,
                mcs_table=My_Config.Ue[0].McsTable + 1,
                mcs_index=My_Config.Ue[0].Mcs
            ),
            mapping_type='A' if ~My_Config.Ue[0].PuschMappingType else 'B',
            n_size_bwp=My_Config.Sys.BwpNRb,
            n_start_bwp=My_Config.Sys.BwpRbOffset,
            num_layers=My_Config.Ue[0].NLayers,
            num_antenna_ports=len(My_Config.Ue[0].DmrsPortSetIdx),
            precoding='non-codebook' if ~My_Config.Ue[0].CodeBookBased else 'codebook',
            tpmi=My_Config.Ue[0].Tpmi,
            transform_precoding=False if ~My_Config.Ue[0].TransformPrecoding else True,
            n_rnti=My_Config.Ue[0].Rnti,
            symbol_allocation=[My_Config.Ue[0].FirstSymb,My_Config.Ue[0].NPuschSymbAll]
        )
    @property
    def first_resource_block(self):
        """
        :class:`~sionna.nr.CarrierConfig` : Carrier configuration
        """
        return self.My_Config.Ue[0].FirstPrb
    
    @property
    def first_subcarrier(self):
        """
        :class:`~sionna.nr.CarrierConfig` : Carrier configuration
        """
        return 12*self.first_resource_block
    
    @property
    def num_resource_blocks(self):
        """
        int, read-only : Number of allocated resource blocks for the
            PUSCH transmissions.
        """
        return self.My_Config.Ue[0].NPrb

    @property
    def dmrs_grid(self):
        # pylint: disable=line-too-long
        """
        complex, [num_dmrs_ports, num_subcarriers, num_symbols_per_slot], read-only : Empty
            resource grid for each DMRS port, filled with DMRS signals

            This property returns for each configured DMRS port an empty
            resource grid filled with DMRS signals as defined in
            Section 6.4.1.1 [3GPP38211]. Not all possible options are implemented,
            e.g., frequency hopping and transform precoding are not available.

            This property provides the *unprecoded* DMRS for each configured DMRS port.
            Precoding might be applied to map the DMRS to the antenna ports. However,
            in this case, the number of DMRS ports cannot be larger than the number of
            layers.
        """
        # Check configuration
        self.check_config()

        # Configure DMRS ports set if it has not been set
        reset_dmrs_port_set = False
        if len(self.dmrs.dmrs_port_set)==0:
            self.dmrs.dmrs_port_set = list(range(self.num_layers))
            reset_dmrs_port_set = True

        # Generate empty resource grid for each port
        a_tilde = np.zeros([len(self.dmrs.dmrs_port_set),
                            self.num_subcarriers,
                            self.carrier.num_symbols_per_slot],
                            dtype=complex)
        first_subcarrier = self.first_subcarrier
        num_subcarriers = self.num_subcarriers

        # For every l_bar
        for l_bar in self.l_bar:

            # For every l_prime
            for l_prime in self.l_prime:

                # Compute c_init
                l = l_bar + l_prime
                c_init = self.c_init(l)
                # Generate RNG
                c = generate_prng_seq(first_subcarrier + num_subcarriers, c_init=c_init)
                c = c[first_subcarrier:]

                # Map to QAM
                r = 1/np.sqrt(2)*((1-2*c[::2]) + 1j*(1-2*c[1::2]))

                # For every port in the dmrs port set
                for j_ind, _ in enumerate(self.dmrs.dmrs_port_set):

                    # For every n
                    for n in self.n:

                        # For every k_prime
                        for k_prime in [0, 1]:

                            if self.dmrs.config_type==1:
                                k = 4*n + 2*k_prime + \
                                    self.dmrs.deltas[j_ind]
                            else: # config_type == 2
                                k = 6*n + k_prime + \
                                    self.dmrs.deltas[j_ind]

                            a_tilde[j_ind, k, self.l_ref+l] = \
                                r[2*n + k_prime] * \
                                self.dmrs.w_f[k_prime][j_ind] * \
                                self.dmrs.w_t[l_prime][j_ind]

        # Amplitude scaling
        a = self.dmrs.beta*a_tilde

        # Reset DMRS port set if it was not set
        if reset_dmrs_port_set:
            self.dmrs.dmrs_port_set = []

        return a
    
Pusch_Config = MyPUSCHConfig(My_Config)
Pusch_Config.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 246
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
My_Config : MyConfig(Sys=SystemConfig(NCellId=246, FrequencyRange=1, BandWidth=100, Numerology=1, CpType=0, NTxAnt=1, NRxAnt=8, BwpNRb=273, BwpRbOffset=0, harqProcFlag=0, nHarqProc=1, rvSeq=0), Ue=[UeConfig(TransformPrecoding=0, Rnti=20002, nId=246, CodeBookBased=0, DmrsPortSetIdx=[0], NLayers=1, NumDmrsCdmGroupsWithoutData=2, Tpmi=0, FirstSymb=0, NPuschSymbAll=14, RaType=1, FirstPrb=31, NPrb=4, FrequencyHoppingMode=0, McsTable=0, Mcs=3, ILbrm=0, nScId=0, NnScIdId=246, DmrsConfigurationType=0, DmrsDuration=1, DmrsAdditionalPosition=1, PuschMappingType=0, DmrsTypeAPositi

In [6]:
"""
frame_number, n_cell_id: No change
n_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)
slot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs
"""

'\nframe_number, n_cell_id: No change\nn_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)\nslot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs\n'

In [52]:
PuschRecord = namedtuple("PuschRecord", [
    "nSFN", "nSlot", "nPDU", "nGroup", "nUlsch", "nUlcch", "nRachPresent",
    "nRNTI", "nUEId", "nBWPSize", "nBWPStart", "nSubcSpacing", "nCpType", "nULType",
    "nMcsTable", "nMCS", "nTransPrecode", "nTransmissionScheme", "nNrOfLayers",
    "nPortIndex", "nNid", "nSCID", "nNIDnSCID", "nNrOfAntennaPorts",
    "nVRBtoPRB", "nPMI", "nStartSymbolIndex", "nNrOfSymbols", "nResourceAllocType",
    "nRBStart", "nRBSize", "nTBSize", "nRV", "nHARQID", "nNDI", "nMappingType",
    "nDMRSConfigType", "nNrOfCDMs", "nNrOfDMRSSymbols", "nDMRSAddPos",
    "nPTRSPresent", "nAck", "nAlphaScaling", "nBetaOffsetACKIndex", "nCsiPart1",
    "nBetaOffsetCsiPart1Index", "nCsiPart2", "nBetaOffsetCsiPart2Index",
    "nTpPi2BPSK", "nTPPuschID", "nRxRUIdx", "nUE", "nPduIdx",

    # New fields for channel and filenames
    "Dmrs_mask",
    "Channel_model", "Speed", "Delay_spread", "Esno_db",
    "Data_filename","Data_dirname"
    ]
)

def save_pickle(data, filename):
    """Saves data to a pickle file."""
    with open(filename, "wb") as f:
        pickle.dump(data, f)

def save_hdf5(data, filename, groupname):
    b, c, y = data
    with h5py.File(filename, "a") as f:
        # sample_num = len(f.keys())  # Ensure unique dataset names
        grp = f.create_group(f"{groupname}")
        grp.create_dataset("b", data=b)
        grp.create_dataset("c", data=c)
        grp.create_dataset("y", data=y)

In [53]:
class MySimulator():
    def __init__(self, pusch_config: MyPUSCHConfig, Rx_included=False):

        self.Num_rx = pusch_config.My_Config.Num_rx
        self.Num_tx = pusch_config.My_Config.Num_tx
    
        tb_size = pusch_config.tb_size
        num_coded_bits = pusch_config.num_coded_bits
        target_coderate = pusch_config.tb.target_coderate
        num_bits_per_symbol = pusch_config.tb.num_bits_per_symbol

        num_layers = pusch_config.num_layers
        n_rnti = pusch_config.n_rnti
        n_id = pusch_config.tb.n_id

        self.Binary_Source = BinarySource(dtype=tf.float32)
        self.TB_Encoder = TBEncoder(target_tb_size=tb_size,
                            num_coded_bits=num_coded_bits,
                            target_coderate=target_coderate,
                            num_bits_per_symbol=num_bits_per_symbol,
                            num_layers=num_layers,
                            n_rnti=n_rnti,
                            n_id=n_id,
                            channel_type="PUSCH",
                            codeword_index=0,
                            use_scrambler=True,
                            verbose=False,
                            output_dtype=tf.float32)
        
        self.Constellation_Mapper = Mapper("qam", num_bits_per_symbol, dtype=tf.complex64)

        self.Layer_Mapper = LayerMapper(num_layers=num_layers, dtype=tf.complex64)
    
        self.Pilot_Pattern = PUSCHPilotPattern([pusch_config], dtype=tf.complex64)

        num_subcarriers = pusch_config.num_subcarriers
        subcarrier_spacing = pusch_config.carrier.subcarrier_spacing*1e3
        fft_size = num_subcarriers
        cp_length = 48
        guard_subcarriers = (0,0)
        # Define the resource grid.
        resource_grid = ResourceGrid(
            num_ofdm_symbols=14,
            fft_size=fft_size,
            subcarrier_spacing=subcarrier_spacing,
            num_tx=self.Num_tx,
            num_streams_per_tx=1,
            cyclic_prefix_length=cp_length,
            num_guard_carriers=guard_subcarriers,
            dc_null=False,
            pilot_pattern=self.Pilot_Pattern,
            dtype=tf.complex64
        )

        self.Resource_Grid_Mapper = ResourceGridMapper(resource_grid, dtype=tf.complex64)        
        
        self.AWGN = AWGN()

        if Rx_included:
            self.Channel_Estimator = PUSCHLSChannelEstimator(
                            resource_grid,
                            pusch_config.dmrs.length,
                            pusch_config.dmrs.additional_position,
                            pusch_config.dmrs.num_cdm_groups_without_data,
                            interpolation_type='nn',
                            dtype=tf.complex64)

            rxtx_association = np.ones([self.Num_rx, self.Num_tx], bool)
            stream_management = StreamManagement(rxtx_association, pusch_config.num_layers)
            self.Mimo_Detector = LinearDetector("lmmse", "bit", "maxlog", resource_grid, stream_management,
                                        "qam", pusch_config.tb.num_bits_per_symbol, dtype=tf.complex64)
            

            self.Layer_Demapper = LayerDemapper(self.Layer_Mapper, num_bits_per_symbol=num_bits_per_symbol)
            self.TB_Decode = TBDecoder(self.TB_Encoder, output_dtype=tf.float32)

        self.Rx_included = Rx_included
        self.tb_size = tb_size
        self.resource_grid = resource_grid
        self.pusch_config = pusch_config
        
    def update_pilots(self, pilots):
        self.Resource_Grid_Mapper._resource_grid.pilot_pattern.pilots = pilots
        if self.Rx_included:
            self.Channel_Estimator = PUSCHLSChannelEstimator(
                            self.Resource_Grid_Mapper._resource_grid,
                            self.pusch_config.dmrs.length,
                            self.pusch_config.dmrs.additional_position,
                            self.pusch_config.dmrs.num_cdm_groups_without_data,
                            interpolation_type='nn',
                            dtype=tf.complex64)

            rxtx_association = np.ones([self.Num_rx, self.Num_tx], bool)
            stream_management = StreamManagement(rxtx_association, self.pusch_config.num_layers)
            self.Mimo_Detector = LinearDetector("lmmse", "bit", "maxlog", self.Resource_Grid_Mapper._resource_grid, stream_management,
                                        "qam", self.pusch_config.tb.num_bits_per_symbol, dtype=tf.complex64)

    def sim(self,channel_model, no_scaling, from_binary_source=True, gen_seed=2004*10+4):
        if from_binary_source:
            b = self.Binary_Source([1, self.Num_tx, self.tb_size])
        else:
            b = tf.reshape(tf.constant(generate_prng_seq(self.Num_tx * self.tb_size, gen_seed), dtype=tf.float32), [1, self.Num_tx, self.tb_size])
        c = self.TB_Encoder(b)
        x_map = self.Constellation_Mapper(c)
        x_layer = self.Layer_Mapper(x_map)
        x = self.Resource_Grid_Mapper(x_layer)

        y, h = channel_model(x)
        no = no_scaling * tf.math.reduce_variance(y)

        y = self.AWGN([y, no])
        if self.Rx_included:
            h_hat,err_var = self.Channel_Estimator([y, no])
            llr_det = self.Mimo_Detector([y, h_hat, err_var, no])
            llr_layer = self.Layer_Demapper(llr_det)

            b_hat, tb_crc_status = self.TB_Decode(llr_layer)

            return tb_crc_status, b, c, y
        
        return b, c, y
    
ue_antenna = Antenna(polarization="single",
                polarization_type="V",
                antenna_pattern="38.901",
                carrier_frequency=2.55e9)

gnb_array = AntennaArray(num_rows=1,
                        num_cols=8//2,
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=2.55e9)

channel_model = CDL(model = 'C',
                            delay_spread = 150*1e-9,
                            carrier_frequency = 2.55e9,
                            ut_array = ue_antenna,
                            bs_array = gnb_array,
                            direction = 'uplink',
                            min_speed = 1,
                            max_speed = 1)
simulator = MySimulator(Pusch_Config)

channel = OFDMChannel(channel_model=channel_model, resource_grid=simulator.resource_grid, 
                                    add_awgn=False, normalize_channel=True, return_channel=True)

b, c, y = simulator.sim(channel, 1., False, 20044)

In [ ]:
def generate_data(name: str,
        data_dir: str,
        pusch_configs: List[MyPUSCHConfig],
        channel_scenarios: List[str],
        esno_dbs: List[float],
        slots: List[int],
        save_dataset=True):
    
    """set up save directory"""
    pusch_records=[]
    parquet_dir = f'{data_dir}/parquet'
    if save_dataset: os.makedirs(parquet_dir, exist_ok=True)
    hdf5_dir = f'{data_dir}/hdf5'
    if save_dataset: os.makedirs(hdf5_dir, exist_ok=True)
    
    """set up for per slot"""
    len_per_case = len(slots)

    total_iterations = len(pusch_configs) * len(channel_scenarios) * len(esno_dbs) * len_per_case


    """generate ..."""
    with tqdm(total=total_iterations, desc="Generating Data") as pbar:
        for config_idx, pusch_config in enumerate(pusch_configs):
            # num_rx = pusch_config.My_Config.Num_rx # 1
            # num_tx = pusch_config.My_Config.Num_tx # 1
            # num_streams_per_tx = pusch_config.num_antenna_ports # 1

            Pusch_Pilot_Set = []
            Pusch_Slots = [4,5,14,15]
            for n, slot in enumerate(Pusch_Slots):
                pusch_config.carrier.slot_number = slot
                pilot_pattern_i = PUSCHPilotPattern([pusch_config], dtype=tf.complex64)
                Pusch_Pilot_Set.append(pilot_pattern_i.pilots)
            len_pusch = len(Pusch_Slots)

            
            """set up for all case"""
            carrier_frequency = pusch_config.My_Config.Carrier_frequency
            num_rx_ant = pusch_config.My_Config.Sys.NRxAnt

            ue_antenna = Antenna(polarization="single",
                            polarization_type="V",
                            antenna_pattern="38.901",
                            carrier_frequency=carrier_frequency)

            gnb_array = AntennaArray(num_rows=1,
                                    num_cols=num_rx_ant//2,
                                    polarization="dual",
                                    polarization_type="cross",
                                    antenna_pattern="38.901",
                                    carrier_frequency=carrier_frequency)
            
            
            simulator = MySimulator(pusch_config)
        
            

            for channel_scenario in channel_scenarios:
                """channel_scenario form: [Normalized]-{CDL}-{A|B|C|D|E}-{delay_spread (ns)}-{speed (m/s)}
                                    or [Normalized]-[OnPL]-[OnSF]-{Umi|Uma}-{low|high}-{delay_spread (ns)}-{speed (m/s)}
                                    
                    Ex: Normalized-CDL-A-150-10"""
                
                chn_scn = channel_scenario.split('-')
                # print(chn_scn)
                normalized = True if chn_scn[0] == 'Normalized' else False # Normalize channel power
                enable_pl = True if chn_scn[1] == 'OnPL' else False # Umi/Uma enable pathloss
                enable_sf = True if chn_scn[2] == 'OnSF' else False # Umi/Uma enable shadow fading
                speed = float(chn_scn[-1])
                delay_spread = float(chn_scn[-2])
                model = chn_scn[-3]
                channel = chn_scn[-4]

                if 'CDL' == channel:
                    channel_model_i = CDL(model = model,
                                            delay_spread = delay_spread,
                                            carrier_frequency = carrier_frequency,
                                            ut_array = ue_antenna,
                                            bs_array = gnb_array,
                                            direction = "uplink",
                                            min_speed = speed)

                else:
                    if 'Umi' == channel:
                        channel_model_i = UMi(carrier_frequency = carrier_frequency,
                                            o2i_model = model,
                                            ut_array = ue_antenna,
                                            bs_array = gnb_array,
                                            direction = "uplink",
                                            enable_pathloss = enable_pl,
                                            enable_shadow_fading = enable_sf)
                    elif 'Uma' == channel:
                        channel_model_i = UMa(carrier_frequency = carrier_frequency,
                                            o2i_model = model,
                                            ut_array = ue_antenna,
                                            bs_array = gnb_array,
                                            direction = "uplink",
                                            enable_pathloss = enable_pl,
                                            enable_shadow_fading = enable_sf)
                
                channel_i = OFDMChannel(channel_model=channel_model_i, resource_grid=simulator.resource_grid, 
                                        add_awgn=False, normalize_channel=normalized, return_channel=True)
                for esno_db in esno_dbs:
                    no_scaling = pow(10., -esno_db / 10.)
                    if channel in ['Umi', 'Uma']:
                        channel_i._cir_sampler.set_topology(*gen_topology(1,1,channel.lower(),min_ut_velocity=speed, max_ut_velocity=speed))

                    for n,slot in enumerate(slots):
                        status_str = f"(config {config_idx} | channel {channel_scenario} | {esno_db} dB | slot {slot} | Sample: {n+1}/{len_per_case}"
                        pbar.set_description(status_str)

                        simulator.update_pilots(Pusch_Pilot_Set[n%len_pusch])
                        b, c, y = simulator.sim(channel_i, no_scaling)

                        assert b.shape[0] == b.shape[1] == c.shape[0] == c.shape[1] == y.shape[0] == y.shape[1] == 1
                        b = tf.cast(b, dtype=tf.uint8)[0][0]
                        c = tf.cast(c, dtype=tf.uint8)[0][0]
                        y =  y[0][0]
                        
                        
                   

                        timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")

                        if save_dataset:
                            save_hdf5([b,c,y], f"{hdf5_dir}/{name}.hdf5", timestamp)
                            # save_pickle(b[0], f'{pickle_dir}/{timestamp}.b.pkl')
                            # save_pickle(c[0], f'{pickle_dir}/{timestamp}.c.pkl')
                            # save_pickle(y[0], f'{pickle_dir}/{timestamp}.y.pkl')

                        pusch_records.append(PuschRecord(
                                            nSFN=(n // len_pusch) % 1023,
                                            nSlot=slot,
                                            nPDU=1,
                                            nGroup=1,
                                            nUlsch=1,
                                            nUlcch=0,
                                            nRachPresent=0,
                                            nRNTI=pusch_config.n_rnti,
                                            nUEId=0,
                                            nBWPSize=pusch_config.n_size_bwp,
                                            nBWPStart=pusch_config.n_start_bwp,
                                            nSubcSpacing=pusch_config.carrier.mu,
                                            nCpType=pusch_config.My_Config.Sys.CpType,
                                            nULType=0,
                                            nMcsTable=pusch_config.tb.mcs_table - 1,
                                            nMCS=pusch_config.tb.mcs_index,
                                            nTransPrecode=pusch_config.My_Config.Ue[0].TransformPrecoding,
                                            nTransmissionScheme=pusch_config.My_Config.Ue[0].CodeBookBased,
                                            nNrOfLayers=pusch_config.num_layers,
                                            nPortIndex=pusch_config.dmrs.dmrs_port_set,
                                            nNid=pusch_config.tb.n_id,
                                            nSCID=pusch_config.dmrs.n_scid,
                                            nNIDnSCID=pusch_config.dmrs.n_id[0],
                                            nNrOfAntennaPorts=pusch_config.My_Config.Sys.NRxAnt,
                                            nVRBtoPRB=0,
                                            nPMI=pusch_config.My_Config.Ue[0].Tpmi,
                                            nStartSymbolIndex=pusch_config.symbol_allocation[0],
                                            nNrOfSymbols=pusch_config.symbol_allocation[1],
                                            nResourceAllocType=1,
                                            nRBStart=pusch_config.first_resource_block,
                                            nRBSize=pusch_config.num_resource_blocks,
                                            nTBSize=(pusch_config.tb_size//8),
                                            nRV=pusch_config.My_Config.Sys.rvSeq,
                                            nHARQID=n % 16,
                                            nNDI=1,
                                            nMappingType=pusch_config.My_Config.Ue[0].PuschMappingType,
                                            nDMRSConfigType=pusch_config.My_Config.Ue[0].DmrsConfigurationType,
                                            nNrOfCDMs=pusch_config.dmrs.num_cdm_groups_without_data,
                                            nNrOfDMRSSymbols=pusch_config.dmrs.length,
                                            nDMRSAddPos=pusch_config.dmrs.additional_position,
                                            nPTRSPresent=pusch_config.My_Config.Ue[0].Ptrs,
                                            nAck=pusch_config.My_Config.Ue[0].OAck,
                                            nAlphaScaling=pusch_config.My_Config.Ue[0].ScalingFactor,
                                            nBetaOffsetACKIndex=pusch_config.My_Config.Ue[0].IHarqAckOffset,
                                            nCsiPart1=pusch_config.My_Config.Ue[0].OCsi1,
                                            nBetaOffsetCsiPart1Index=pusch_config.My_Config.Ue[0].ICsi1Offset,
                                            nCsiPart2=pusch_config.My_Config.Ue[0].OCsi2,
                                            nBetaOffsetCsiPart2Index=pusch_config.My_Config.Ue[0].ICsi2Offset,
                                            nTpPi2BPSK=pusch_config.My_Config.Ue[0].TpPi2Bpsk,
                                            nTPPuschID=pusch_config.My_Config.Ue[0].NRsId,
                                            nRxRUIdx=np.arange(0, pusch_config.My_Config.Sys.NRxAnt),
                                            nUE=1,
                                            nPduIdx=[0],
                                            Dmrs_mask=pusch_config.dmrs_mask.T.flatten(),
                                            Channel_model=f"{chn_scn[:-2]}",
                                            Speed=speed,
                                            Delay_spread=delay_spread,
                                            Esno_db=esno_db,
                                            Data_filename=timestamp,
                                            Data_dirname=name
                                        )
                                    )
                        pbar.update(1)  # Increment progress

                df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
                if save_dataset: df.to_parquet(f'{parquet_dir}/{name}.parquet', engine="pyarrow")
    return df

In [ ]:
# samples_per_case = 1

data_dir = '../Pusch_data/dataset'
# os.makedirs(data_dir,exist_ok=True)
# os.makedirs(f'{data_dir}/pickle',exist_ok=True)
# os.makedirs(f'{data_dir}/parquet',exist_ok=True)
Pusch_Config_2 = Pusch_Config.clone()
Pusch_Config_2.n_rnti = 10005
Pusch_Config_3 = Pusch_Config.clone()
Pusch_Config_3.n_rnti = 12404
Pusch_Config_3.My_Config.Ue[0].FirstPrb = 100
name = datetime.now().strftime("%Y%m%d%H%M%S%f")
df = generate_data(name=f'{name}',
              data_dir=data_dir,
              pusch_configs=[Pusch_Config, Pusch_Config_2, Pusch_Config_3],
              channel_scenarios=['CDL-C-150-10'],
              esno_dbs=[i for i in np.arange(5,5.1,5)],
              slots=[4,5,14,15],
              save_dataset=True
)

Generating Data:   0%|          | 0/12 [00:00<?, ?it/s]

In [72]:
df.iloc[0]

nSFN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        0
nSlot                                                                                                                                                                                                                                                                     

In [179]:
# drive_parquet_folder_id = "1p9e9tw9iFrXiaFFh_TP-cuaHwxwLYMYb"
# drive_pickle_folder_id = "1_kx1wUO8eBe7tltrYQhWWNFhS8dbfDFW"

# def create_drive_folder(folder_name, parent_folder_id=None):
#     """Creates a folder in Google Drive and returns the folder ID."""
#     folder_metadata = {
#         "name": folder_name,
#         "mimeType": "application/vnd.google-apps.folder",
#     }
#     if parent_folder_id:
#         folder_metadata["parents"] = [parent_folder_id]

#     folder = drive_service.files().create(body=folder_metadata, fields="id").execute()
#     print(f'Folder "{folder_name}" created with ID: {folder["id"]}')
#     return folder["id"]

# def upload_files_to_drive(data_dir, name, folder_ids):
#     files_to_upload = [f'{data_dir}/parquet/{name}.parquet']
#     for (root, _, file) in os.walk(f'{data_dir}/pickle/{name}'):
#         for f in file:
#             files_to_upload.append(os.path.join(root, f))

#     pickle_subfolder_id = create_drive_folder(name, folder_ids[1])
#     """Uploads multiple files sequentially but with optimizations."""
#     for file_path in tqdm(files_to_upload, desc="Uploading Files"):
#         file_name = file_path.split("/")[-1]
#         folder_id = folder_ids[0] if ".parquet" in file_name else pickle_subfolder_id

#         file_metadata = {"name": file_name, "parents": [folder_id]}
#         media = MediaFileUpload(file_path, resumable=True)  # Enables resumable uploads

#         try:
#             drive_service.files().create(body=file_metadata, media_body=media, fields="id").execute()
#         except Exception as e:
#             print(f"Error uploading {file_name}: {e}")

#     # if os.path.exists(data_dir): os.rename(data_dir, '../Dataset/'+datetime.now().strftime('%Y%m%d%H%M'))
# # Example Usage
# data_dir = '../Dataset/tmp'
# upload_filename = name
# upload_files_to_drive(data_dir, upload_filename, [drive_parquet_folder_id, drive_pickle_folder_id])


Hopefully you have enjoyed this tutorial on Sionna's 5G NR PUSCH module!

Please have a look at the [API documentation](https://nvlabs.github.io/sionna/api/sionna.html) of the various components or the other available [tutorials](https://nvlabs.github.io/sionna/tutorials.html) to learn more.